In [28]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import time
from selenium import webdriver
import os
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [26]:
class Zarascrape:

    def __init__ (self, url): # Declaro las variables self de la clase
        self.url = url 
        self.col_names = ["url", "description"]
        self.product_list = [] 
 
    def get_info(self, url): # saca la info de cada vestido en particular
        html = requests.get(url).content
        data = BeautifulSoup(html, "lxml")
        prod_title=data.find("h1",{"class":"product-name"})
        title=prod_title.text.replace(" Detalle","")
        prod_descr = data.find("p",{"class":"description"})
        prod_various = data.find("p",{"class":"product-color"})
        prod_color=prod_various.find("span",{"class": "_colorName"})
        prod_ref=prod_various.find("span",{"data-qa-qualifier": "product-reference"})
        prod_price=data.find_all("script",{"type":"application/ld+json"})
        for i in prod_price:
            price=i.text.split(',')[-2].lstrip('"price": ').replace('"','')
        prod_size=data.find_all("div",{"class":"size-select"})
        for all_size in prod_size:
            size=all_size.find_all("label", {'class':'product-size _product-size'}) #filtramos por tallas disponibles
            available_size=[s.text for s in size]
        prod_images=data.find_all("a",{"class":"_seoImg main-image"})
        prod_image=[e.get("href") for e in prod_images][0].lstrip("//")
        try:
            features=[title,prod_descr.text,prod_color.text,prod_ref.text, price, available_size,prod_image]
        except:
            features=["","","","","","",""]
        return features

    def get_attrs(webelement, attr='href'): 
        try: 
            return webelement.get_attribute(attr)
        except: 
            return ''
    
    def get_list(self): # List of urls para vestidos
        driver = webdriver.Chrome((ChromeDriverManager().install()))
        driver.get(self.url)
        source_data = driver.page_source
        elem = driver.find_element_by_tag_name("body")

        no_of_pagedowns = 10

        res = []

        while no_of_pagedowns:
            elem.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
            no_of_pagedowns-=1
        div = driver.find_elements_by_class_name('product-info-item')
        for i in div:
            b=i.find_elements_by_css_selector('a')
            for c in b:
                res.append(c)
        urls = [get_attrs(e, 'href') for e in res]
        return urls

    def loop_it(self): # Iterates over the url root. 
        iterable = self.get_list()
        for e in iterable:
            self.product_list.append(self.get_info(e))
            time.sleep(3)
        return self.product_list
    
url_vestidos = 'https://www.zara.com/uk/en/woman-dresses-l1066.html?v1=1180427' # Scrapea sólo vestidos

# Assigns the Zarascrape class for dresses to an object
vestidos_ls = Zarascrape(url_vestidos)

# Launches the class method
data_vestidos = vestidos_ls.loop_it()

print(len(data_vestidos))
data_vestidos[:5]


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/almu/.wdm/chromedriver/2.46/mac64/chromedriver
632


[['LACE DRESS Details',
  'Short sleeve dress with a shirt collar. Featuring matching interior lining and metal button fastening in the front. HEIGHT OF MODEL: 177 cm. / 5′ 9″',
  'White',
  '1639/056',
  '39.99',
  ['XS', 'S', 'M'],
  'static.zara.net/photos///2019/V/0/1/p/1639/056/250/2/w/560/1639056250_1_1_1.jpg?ts=1552560925886'],
 ['DRESS WITH MATCHING EMBROIDERY Details',
  'V-neck dress with sleeves falling below the elbow. Featuring an elastic interior waistband, side in-seam pockets, lining, matching embroidered appliqués on the top and a tied opening in the back. HEIGHT OF MODEL: 177 cm. / 5′ 9″',
  'White',
  '1821/021',
  '25.99',
  ['XS', 'S', 'M', 'L'],
  'static.zara.net/photos///2019/V/0/1/p/1821/021/250/2/w/560/1821021250_1_1_1.jpg?ts=1552564646334'],
 ['EMBROIDERED LACE DRESS Details',
  'Round neck dress with sleeves reaching below the elbow. Featuring a lace trim detail on the front, same-colour embroidered appliqués, a ruffled hem and front button fastening. HEIGHT

In [29]:
# Saves the df into a .csv
zara_base = pd.DataFrame(data_vestidos)

In [35]:
zara_base.head()

,0,1,2,3,4,5,6
0,LACE DRESS Details,Short sleeve dress with a shirt collar. Featur...,White,1639/056,39.99,"[XS, S, M]",static.zara.net/photos///2019/V/0/1/p/1639/056...
1,DRESS WITH MATCHING EMBROIDERY Details,V-neck dress with sleeves falling below the el...,White,1821/021,25.99,"[XS, S, M, L]",static.zara.net/photos///2019/V/0/1/p/1821/021...
2,EMBROIDERED LACE DRESS Details,Round neck dress with sleeves reaching below t...,Ecru,5770/022,29.99,"[XS, S, M, L]",static.zara.net/photos///2019/V/0/1/p/5770/022...
3,CROCHET DRESS Details,Dress with ruffled straps. HEIGHT OF MODEL: 17...,Black / Brown,3653/004,39.99,"[S, M]",static.zara.net/photos///2019/V/0/1/p/3653/004...
4,LINEN DRESS WITH SEASHELL TRIMS Details,Slot collar dress with an adjustable tie at th...,Black,2161/644,79.99,"[XS, S, M, L, XL]",static.zara.net/photos///2019/V/0/1/p/2161/644...


In [31]:
zara_base.shape

(632, 7)

In [36]:
# Saves the df into a .csv
zara_dresses = zara_base.to_csv("zara.csv", index=False)

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19      True
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
602     True
603     True
604     True
605     True
606    False
607     True
608     True
609     True
610    False
611     True
612     True
613    False
614    False
615     True
616     True
617    False
618    False
619     True
620    False
621    False
622    False
623     True
624     True
625    False
626     True
627    False
628    False
629     True
630     True
631    False
Length: 632, dtype: bool